# Overfitting

### Imports and Dataset

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [2]:
df = pd.read_pickle('../training_dataset_task3/task_3_training_e8da4715deef7d56_f8b7378_pandas.pkl').reset_index()

# only low and mid level features in X
X = df.loc[:, 'essentia_dissonance_mean':'mirtoolbox_roughness_pct_90']
y = df['quadrant']

# preprocess dataset
X_std = StandardScaler().fit_transform(X)
X = pd.DataFrame(X_std, columns=X.columns)

### Cross Validation - hold out on segment 26, used for final testing

In [3]:
from sklearn.model_selection import cross_val_score

# add segment ID to remove segment 26
X["segment_id"] = df['segment_id']
seg_26_indices = (X['segment_id'] == 26)

X_test = X[seg_26_indices].drop(['segment_id'], axis=1)
y_test = y[seg_26_indices]

X_train = X.drop(X[seg_26_indices].index, axis=0).reset_index(drop=True)
y_train = y.drop(X[seg_26_indices].index, axis=0)

In [4]:
base_model = RandomForestRegressor(n_estimators=10, random_state=42)
# base_cv = cross_val_score(base_model, X_train, y_train, cv=10)

# print('Cross Validation on base model gives: ', base_cv.mean())

In [5]:
best_gs = RandomForestRegressor(n_estimators=800, max_depth=50, max_features='sqrt', min_samples_leaf=2, bootstrap=False)
# best_cv = cross_val_score(best_gs, X_train, y_train, cv=2)

# print('Cross Validation on best grid search model gives: ', best_cv.mean())

## Overfitting

In [6]:
def evaluate(model, X_test, y_test):
    predictions = model.predict(X_test)
    errors = abs(predictions - y_test)
    m = 100 * np.mean(errors / y_test)
    accuracy = 100 - m
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    return accuracy

### high number of n_estimators

In [7]:
of_estimators = RandomForestRegressor(n_estimators=1500)
# of_estimators.fit(X_train, y_train)

# score on training set:
print('Score on Training Set')
# evaluate(of_estimators, X_train, y_train)

# score on test segment:
print('\nScore on Test Set')
# evaluate(of_estimators, X_test, y_test)

Score on Training Set

Score on Test Set


### high number of max_depth

In [8]:
of_depth = RandomForestRegressor(max_depth=1000)
# of_depth.fit(X_train, y_train)

# score on training set:
print('Score on Training Set')
# evaluate(of_depth, X_train, y_train)

# score on test segment:
print('\nScore on Test Set')
# evaluate(of_depth, X_test, y_test)

Score on Training Set

Score on Test Set


In [9]:
best_gs.fit(X_train,y_train)
evaluate(best_gs, X_test, y_test)

C:\Users\rubin\miniconda3\envs\py\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names seen at fit time, yet now missing:
- segment_id

  warnings.warn(message, FutureWarning)


ValueError: X has 169 features, but RandomForestRegressor is expecting 170 features as input.